In [2]:
import pandas as pd
import numpy as np
import traceback

from esa_snappy import ProductIO
from esa_snappy import GeoPos
from esa_snappy import PixelPos

from glob import glob
from tqdm import tqdm
import os
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific warning messages
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="org.esa.snap.core.dataio.dimap.DimapProductReader")


INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.


In [3]:

def check_image(mgrs):
    processed_mosaic = os.listdir("/data/ksa/01_Image_Acquisition/02_Processed_mosaic/")
    if(mgrs in processed_mosaic):
        return True
    else: 
        return False

def get_gc(path_to_sentinel_data):
    # path_to_sentinel_data = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/48MXU/20230501_20230512.dim"
    product_subset = ProductIO.readProduct(path_to_sentinel_data)
    gc = product_subset.getSceneGeoCoding()
    bands_names = list(product_subset.getBandNames())
    return product_subset,gc,bands_names

def get_values(gc, bands_names, lat, long):
    pixel_pos = gc.getPixelPos(GeoPos(lat, long), None)
    data = list()
    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
            data_values = [float(val) for val in data]
        except Exception as e:
            #print('error occured at',int(pixel_pos.x), int(pixel_pos.y))
            #print(int(pixel_pos.x), int(pixel_pos.y))
            continue
    return data_values

# data_values
def get_value(gc, lat, long):    
    
    data = list()
    pixel_pos = gc.getPixelPos(GeoPos(lat, lon), None)
    data.append(lat).append(lon).append(int(pixel_pos.x)).append(int(pixel_pos.y))

    for i, band_name in enumerate(bands_names):
        temp_band = product_subset.getBand(band_name)
        width, height = temp_band.getRasterWidth(), temp_band.getRasterHeight()
        try:
            tmp = np.zeros(1)
            temp_band.readPixels(int(pixel_pos.x), int(pixel_pos.y), 1, 1, tmp)
            data.append(tmp[0])
        except Exception as e:
            print(int(pixel_pos.x), int(pixel_pos.y))
            print(e)
            #traceback.print_exc()
            continue
    return data


In [1]:
## 32 35 16 63 12 73 18 94
## 53 97  

## edit with province
idprov = "94"

In [4]:
df_ksa = pd.read_csv("/data/raw/processed/cloned_points.csv")
# df_ksa.iloc[:1000].to_csv("cek.csv")

# df_ksa = pd.read_csv("/data/raw/processed/generated_points_unstable.csv")
df_ksa['idprov'] = df_ksa.idsegmen.astype('str').str[:2]
df_ksa['idkab'] = df_ksa.idsegmen.astype('str').str[:4]

df_ksa['index'] = [x.zfill(2) for x in df_ksa['index'].astype("str")]
df_ksa['idpoint'] = df_ksa.idsegmen.astype('str') + df_ksa.idsubsegmen.astype('str') + '#' + df_ksa['index']

#df_ksa = df_ksa.query('idprov == "{0}"'.format(idprov))

df_ksa.head()

,iterx,itery,lat,long,index,idsegmen,strati,idsubsegmen,EASTING,NORTHING,100kmSQ_ID,GZD,MGRS,idprov,idkab,idpoint
0,1,1,2.381336,96.481115,01,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#01
1,1,2,2.381156,96.481115,02,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#02
2,1,3,2.380977,96.481115,03,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#03
3,1,4,2.380797,96.481115,04,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#04
4,1,5,2.380618,96.481115,05,110101001,S2,A1,200000mE,200000mN,KC,47N,47NKC,11,1101,110101001A1#05


In [5]:
pivot_ed=df_ksa.pivot_table(
    index='idprov',
    values='idpoint',
    aggfunc='count'
).reset_index()

In [6]:
pivot_ed['idpoint_percent']=pivot_ed.idpoint/pivot_ed.idpoint.sum()*100

In [7]:
x=pivot_ed.query('idprov in ["12","16","18","32","35","53","63","73","94","97"]')

In [8]:
x.idpoint_percent.sum()

np.float64(58.174696435566005)

In [9]:
x.rename(columns={'idpoint':'JLH TITIK KSA','idpoint_percent':'% NASIONAL'})

,idprov,JLH TITIK KSA,% NASIONAL
1,12,360675,6.278888
5,16,289350,5.037211
7,18,217125,3.779867
11,32,584550,10.176263
14,35,1085850,18.903251
18,53,174150,3.031727
21,63,235800,4.104975
26,73,363600,6.329808
33,94,30600,0.532707


In [11]:
## list all MGRS
mgrs_all_ = list(df_ksa.MGRS.unique())

## Cek MGRS not done
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

['47NKC',
 '47NJC',
 '46NHH',
 '47NJD',
 '46NHJ',
 '47NLC',
 '47NMC',
 '47NLD',
 '47NKD',
 '47NLE',
 '47NLF',
 '47NKE',
 '47NKF',
 '47NJE',
 '46NHK',
 '47NJF',
 '46NGL',
 '46NGM',
 '46NHL',
 '46NHM',
 '47NME',
 '47NLB',
 '47NNA',
 '47NPA',
 '47NNB',
 '47NMB',
 '47NNC',
 '47NPC',
 '47NND',
 '47NMD',
 '47NLA',
 '47NNE',
 '47NPB',
 '47NPD',
 '47MPS',
 '47MPT',
 '47MNT',
 '47MNU',
 '47MMU',
 '47MQT',
 '47MQU',
 '47MPU',
 '47MPV',
 '47MQV',
 '47MRU',
 '47MNV',
 '47MRV',
 '48MTE',
 '48MSE',
 '48MUE',
 '47NRA',
 '48NSF',
 '48NUF',
 '48NTF',
 '47NQA',
 '48NSG',
 '47NQB',
 '47NRB',
 '47NQC',
 '48NTG',
 '47MRT',
 '48MSC',
 '48MTC',
 '48MTD',
 '48MUD',
 '48MUC',
 '48MVD',
 '48MSD',
 '48MUA',
 '48MVA',
 '48MWA',
 '48MWB',
 '48MVB',
 '48MWC',
 '48MUB',
 '48MTA',
 '48MTB',
 '48MVC',
 '48MUV',
 '48MVV',
 '48MTV',
 '48MSB',
 '47MRS',
 '47MQS',
 '48MSA',
 '48MVU',
 '48MWU',
 '48MWV',
 '48MXC',
 '48MWD',
 '48MZB',
 '48MYB',
 '48MYC',
 '48MZC',
 '48MXB',
 '49MAS',
 '49MAT',
 '48NVG',
 '49NAE',
 '49NBC',


In [ ]:
list_un_done=[]
for mgrs in mgrs_not_done_:
    df_result = pd.DataFrame()
    if(check_image(mgrs)):
        try:
            print(mgrs + " available")
            path_to_mgrs = "/data/ksa/01_Image_Acquisition/02_Processed_mosaic/"+mgrs+"/"
            all_sentinel_data = glob(path_to_mgrs+"*.dim")
            df_ksa_mgrs = df_ksa.loc[df_ksa.MGRS == mgrs]
            for sent in tqdm(all_sentinel_data):
                path_to_sentinel_data = sent
                periode = path_to_sentinel_data[-21:-4]
                product_subset,gc,bands_names = get_gc(path_to_sentinel_data)
                bands_names = [col for col in bands_names if 'VH' in col]
                df_ksa_mgrs_tmp = df_ksa_mgrs.copy()
                df_ksa_mgrs_tmp = df_ksa_mgrs_tmp.reset_index(drop=True)
                df_ksa_mgrs_tmp.loc[:, bands_names] = 0.0
                df_ksa_mgrs_tmp.loc[:, 'periode'] =periode  
                # print(bands_names)
                for i,r in df_ksa_mgrs_tmp.iterrows():
                    try:
                        val = get_values(gc, bands_names, r['lat'], r['long'])
                        df_ksa_mgrs_tmp.loc[i,bands_names] = val
                    except:
                        print('error while get the data on',r['lat'], r['long'])
                        list_un_done.append([r['lat'], r['long']])
                        continue
                    # break
                # break
                df_result = pd.concat([df_result,df_ksa_mgrs_tmp])
    
            df_result = df_result[['idpoint','MGRS','Sigma0_VH_db','periode']]
            with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'wb') as f:
                pickle.dump(df_result, f)
            
            print("done sampling for : "+ mgrs)
            # break
        except: 
            print("Error on : "+ mgrs)
    else:
        print(mgrs + " not available")
        continue
    # break

47NKC not available
47NJC not available
46NHH not available
47NJD not available
46NHJ not available
47NLC available


 25%|██▌       | 15/60 [03:40<11:31, 15.36s/it]

In [ ]:
df_ksa_mgrs = df_ksa.loc[df_ksa.MGRS == '51MST']
df_ksa_mgrs

In [ ]:
## Cek error mgrs
mgrs_done_ = os.listdir('/data/ksa/03_Sampling/data/'+idprov+'/') 
mgrs_done_ = [x[-9:-4] for x in mgrs_done_]

mgrs_not_done_ = [x for x in mgrs_all_ if x not in mgrs_done_]
mgrs_not_done_

### Cek 

In [ ]:
## Cek VH Value Overtime on One Point

filtered_df = df_.query('idpoint == "321201001A1#02"').sort_values('periode')

plt.figure(figsize=(10, 6))
plt.plot(filtered_df['periode'], filtered_df['Sigma0_VH_db'], marker='o')

plt.title('Sigma0_VH_db over Time for idpoint "321201003B2#13"')
plt.xlabel('Periode')
plt.ylabel('Sigma0_VH_db')
plt.grid(True)
plt.xticks(rotation=45)  # Rotate x-axis labels if needed

plt.tight_layout()
plt.show()

In [ ]:
## Open one Sampling Data

mgrs = '49LDM'
with open('/data/ksa/03_Sampling/data/'+idprov+'/sampling_'+mgrs+'.pkl', 'rb') as f:
    df_ = pickle.load(f)

df_.head()

In [ ]:
df_.describe()